# Feature Engineering and Complex Transformations

This notebook demonstrates advanced feature engineering using PySpark functions on the processed YouTube Analytics data.

## Tasks:
1. **engagement_score**: Weighted metric combining likes, dislikes, and comments relative to views
2. **days_to_trend**: Calculate days between trending_date and publish_time
3. **trending_rank**: Rank videos within each trending_date and category using Window Functions

In [1]:
# Setup notebook environment
from notebook_setup import setup_notebook_environment, test_imports

# Setup paths and test imports
project_root = setup_notebook_environment()
test_imports()

✅ Project root: e:\Study Space\Analytics Enginerring\Data Engineering\Azure Databricks\ADB_Practice\YouTube Analytics
✅ Added to Python path:
   - e:\Study Space\Analytics Enginerring\Data Engineering\Azure Databricks\ADB_Practice\YouTube Analytics
   - e:\Study Space\Analytics Enginerring\Data Engineering\Azure Databricks\ADB_Practice\YouTube Analytics\src
✅ Config import successful
✅ SparkUtils import successful
✅ YouTubeDataReader import successful
✅ All imports working correctly!


True

In [2]:
# Import required modules
from config.settings import Config
from src.utils.spark_utils import SparkUtils
from src.data_ingestion.processed_data_loader import ProcessedDataLoader

# PySpark functions for feature engineering
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, DoubleType

import logging
logging.basicConfig(level=logging.INFO)

In [3]:
# Initialize Spark session
spark = SparkUtils.get_spark_session()
print(f"Spark version: {spark.version}")

# Initialize data loader
loader = ProcessedDataLoader(spark)
config = Config()

Spark version: 3.5.6


## Load Processed Data

Load the processed YouTube trending videos data that was generated by our pipeline.

In [4]:
# Load processed video data
print("Loading processed YouTube trending videos data...")
df = loader.load_processed_videos()

if df is None:
    raise ValueError("Could not load processed data. Please run the pipeline first.")

print(f"Loaded {df.count():,} records")
print("\nSchema:")
df.printSchema()

INFO:src.data_ingestion.processed_data_loader:Attempting to load using Pandas I/O (Windows compatibility)
INFO:src.data_ingestion.processed_data_loader:Loading from pandas parquet: e:\Study Space\Analytics Enginerring\Data Engineering\Azure Databricks\ADB_Practice\YouTube Analytics\data\processed\youtube_trending_videos.parquet
INFO:src.utils.pandas_io:Loading parquet file from e:\Study Space\Analytics Enginerring\Data Engineering\Azure Databricks\ADB_Practice\YouTube Analytics\data\processed\youtube_trending_videos.parquet using Pandas


Loading processed YouTube trending videos data...


INFO:src.utils.pandas_io:Successfully loaded 40899 records from e:\Study Space\Analytics Enginerring\Data Engineering\Azure Databricks\ADB_Practice\YouTube Analytics\data\processed\youtube_trending_videos.parquet
INFO:src.data_ingestion.processed_data_loader:Loaded 40899 processed video records


Loaded 40,899 records

Schema:
root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: long (nullable = true)
 |-- likes: long (nullable = true)
 |-- dislikes: long (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: boolean (nullable = true)
 |-- ratings_disabled: boolean (nullable = true)
 |-- video_error_or_removed: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- country: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- publish_time_parsed: timestamp (nullable = true)



In [5]:
# Show sample data
print("Sample of loaded data:")
df.select(
    "video_id", "title", "channel_title", "trending_date", "publish_time",
    "views", "likes", "dislikes", "comment_count", "category_name", "country"
).show(5, truncate=False)

Sample of loaded data:
+-----------+---------------------------------------------------------+-------------+-------------+------------------------+-------+-----+--------+-------------+---------------+-------+
|video_id   |title                                                    |channel_title|trending_date|publish_time            |views  |likes|dislikes|comment_count|category_name  |country|
+-----------+---------------------------------------------------------+-------------+-------------+------------------------+-------+-----+--------+-------------+---------------+-------+
|-2aVkGcI7ZA|Benedict Cumberbatch's Tom Holland impression is PERFECT.|BBC Radio 1  |18.27.04     |2018-04-25T12:20:45.000Z|1012527|19339|633     |520          |Music          |US     |
|-2b4qSoMnKE|Ex-UFO program chief: We may not be alone                |CNN          |17.20.12     |2017-12-19T20:46:33.000Z|84744  |1444 |199     |1610         |News & Politics|US     |
|-37nIo_tLnk|Christmas Day 2000                

## Task 1: Calculate Engagement Score

Create an `engagement_score` column using the formula:
```
engagement_score = ((likes * 0.5) + (dislikes * 0.2) + (comment_count * 0.3)) / views
```

We'll handle division-by-zero errors by setting engagement_score to 0 when views is 0 or null.

In [6]:
# Task 1: Calculate engagement_score
print("Task 1: Calculating engagement_score...")

# Calculate engagement score with division-by-zero handling
df_with_engagement = df.withColumn(
    "engagement_score",
    F.when(
        (F.col("views").isNull()) | (F.col("views") == 0),
        0.0
    ).otherwise(
        (
            (F.coalesce(F.col("likes"), F.lit(0)) * 0.5) +
            (F.coalesce(F.col("dislikes"), F.lit(0)) * 0.2) +
            (F.coalesce(F.col("comment_count"), F.lit(0)) * 0.3)
        ) / F.col("views")
    ).cast(DoubleType())
)

print("Engagement score calculated successfully!")

# Show sample results
print("\nSample engagement scores:")
df_with_engagement.select(
    "video_id", "title", "views", "likes", "dislikes", "comment_count", "engagement_score"
).orderBy(F.desc("engagement_score")).show(10, truncate=False)

Task 1: Calculating engagement_score...
Engagement score calculated successfully!

Sample engagement scores:
+-----------+-------------------------------------------------------------+-------+-------+--------+-------------+-------------------+
|video_id   |title                                                        |views  |likes  |dislikes|comment_count|engagement_score   |
+-----------+-------------------------------------------------------------+-------+-------+--------+-------------+-------------------+
|LsoLEjrDogU|Bruno Mars - Finesse (Remix) [Feat. Cardi B] [Official Video]|548621 |159356 |2374    |19455        |0.1567371646364248 |
|TyHvyGVs42U|Luis Fonsi, Demi Lovato - Échame La Culpa                    |499946 |135292 |3528    |12094        |0.14397514931612615|
|8O_MwlZ2dEg|j-hope 'Airplane' MV                                         |5275672|1401915|6268    |158127       |0.14209549039439903|
|UaAHl_m_ybk|5 Seconds Of Summer - Want You Back (Audio)                  |439056

In [7]:
# Analyze engagement score statistics
print("Engagement Score Statistics:")
engagement_stats = df_with_engagement.select(
    F.count("engagement_score").alias("count"),
    F.mean("engagement_score").alias("mean"),
    F.stddev("engagement_score").alias("stddev"),
    F.min("engagement_score").alias("min"),
    F.max("engagement_score").alias("max"),
    F.expr("percentile_approx(engagement_score, 0.5)").alias("median")
).collect()[0]

for field in engagement_stats.asDict():
    value = engagement_stats[field]
    if isinstance(value, float):
        print(f"{field}: {value:.6f}")
    else:
        print(f"{field}: {value}")

Engagement Score Statistics:
count: 40899
mean: 0.018867
stddev: 0.014349
min: 0.000000
max: 0.156737
median: 0.015630


## Task 2: Calculate Days to Trend

Calculate the number of days between `trending_date` and `publish_time`.
This requires parsing both date columns and computing the difference.

In [8]:
# Task 2: Calculate days_to_trend
print("Task 2: Calculating days_to_trend...")

# First, let's examine the date formats
print("Examining date formats:")
df_with_engagement.select("trending_date", "publish_time").show(5, truncate=False)

Task 2: Calculating days_to_trend...
Examining date formats:
+-------------+------------------------+
|trending_date|publish_time            |
+-------------+------------------------+
|18.27.04     |2018-04-25T12:20:45.000Z|
|17.20.12     |2017-12-19T20:46:33.000Z|
|17.28.12     |2009-12-15T23:26:32.000Z|
|17.31.12     |2009-12-15T23:26:32.000Z|
|18.25.04     |2018-04-23T12:11:34.000Z|
+-------------+------------------------+
only showing top 5 rows



In [9]:
# Parse trending_date (format: YY.DD.MM) and publish_time (ISO format)
df_with_dates = df_with_engagement.withColumn(
    "trending_date_parsed",
    F.to_date(F.col("trending_date"), "yy.dd.MM")
).withColumn(
    "publish_date_parsed",
    F.to_date(F.col("publish_time_parsed"))
)

# Calculate days_to_trend
df_with_days_to_trend = df_with_dates.withColumn(
    "days_to_trend",
    F.when(
        F.col("trending_date_parsed").isNull() | F.col("publish_date_parsed").isNull(),
        None
    ).otherwise(
        F.datediff(F.col("trending_date_parsed"), F.col("publish_date_parsed"))
    ).cast(IntegerType())
)

print("Days to trend calculated successfully!")

# Show sample results
print("\nSample days_to_trend calculations:")
df_with_days_to_trend.select(
    "video_id", "title", "trending_date", "publish_time", 
    "trending_date_parsed", "publish_date_parsed", "days_to_trend"
).filter(F.col("days_to_trend").isNotNull()).show(10, truncate=False)

Days to trend calculated successfully!

Sample days_to_trend calculations:
+-----------+---------------------------------------------------------+-------------+------------------------+--------------------+-------------------+-------------+
|video_id   |title                                                    |trending_date|publish_time            |trending_date_parsed|publish_date_parsed|days_to_trend|
+-----------+---------------------------------------------------------+-------------+------------------------+--------------------+-------------------+-------------+
|-2aVkGcI7ZA|Benedict Cumberbatch's Tom Holland impression is PERFECT.|18.27.04     |2018-04-25T12:20:45.000Z|2018-04-27          |2018-04-25         |2            |
|-2b4qSoMnKE|Ex-UFO program chief: We may not be alone                |17.20.12     |2017-12-19T20:46:33.000Z|2017-12-20          |2017-12-19         |1            |
|-37nIo_tLnk|Christmas Day 2000                                       |17.28.12     |2009-12-15

In [10]:
# Analyze days_to_trend statistics
print("Days to Trend Statistics:")
days_stats = df_with_days_to_trend.select(
    F.count("days_to_trend").alias("count"),
    F.mean("days_to_trend").alias("mean"),
    F.stddev("days_to_trend").alias("stddev"),
    F.min("days_to_trend").alias("min"),
    F.max("days_to_trend").alias("max"),
    F.expr("percentile_approx(days_to_trend, 0.5)").alias("median")
).collect()[0]

for field in days_stats.asDict():
    value = days_stats[field]
    if isinstance(value, float):
        print(f"{field}: {value:.2f}")
    else:
        print(f"{field}: {value}")

# Show distribution of days_to_trend
print("\nDays to Trend Distribution:")
df_with_days_to_trend.groupBy("days_to_trend").count().orderBy("days_to_trend").show(20)

Days to Trend Statistics:
count: 40899
mean: 16.83
stddev: 146.10
min: 0
max: 4215
median: 5

Days to Trend Distribution:
+-------------+-----+
|days_to_trend|count|
+-------------+-----+
|            0|  121|
|            1| 2839|
|            2| 4284|
|            3| 4639|
|            4| 4680|
|            5| 4579|
|            6| 4038|
|            7| 3126|
|            8| 2184|
|            9| 1657|
|           10| 1370|
|           11| 1198|
|           12| 1082|
|           13|  938|
|           14|  793|
|           15|  608|
|           16|  471|
|           17|  396|
|           18|  312|
|           19|  222|
+-------------+-----+
only showing top 20 rows



## Task 3: Calculate Trending Rank

Use PySpark Window Functions to rank videos within each `trending_date` and `category_name` based on their `views` in descending order.

In [11]:
# Task 3: Calculate trending_rank using Window Functions
print("Task 3: Calculating trending_rank using Window Functions...")

# Define window specification: partition by trending_date and category_name, order by views descending
window_spec = Window.partitionBy("trending_date", "category_name").orderBy(F.desc("views"))

# Calculate trending_rank
df_with_rank = df_with_days_to_trend.withColumn(
    "trending_rank",
    F.row_number().over(window_spec)
)

print("Trending rank calculated successfully!")

# Show sample results
print("\nSample trending ranks (Top 3 videos per category per day):")
df_with_rank.select(
    "trending_date", "category_name", "trending_rank", "title", "channel_title", "views"
).filter(F.col("trending_rank") <= 3).orderBy(
    "trending_date", "category_name", "trending_rank"
).show(20, truncate=False)

Task 3: Calculating trending_rank using Window Functions...
Trending rank calculated successfully!

Sample trending ranks (Top 3 videos per category per day):
+-------------+----------------+-------------+--------------------------------------------------------------------------------------+------------------------------------+--------+
|trending_date|category_name   |trending_rank|title                                                                                 |channel_title                       |views   |
+-------------+----------------+-------------+--------------------------------------------------------------------------------------+------------------------------------+--------+
|17.01.12     |Autos & Vehicles|1            |The Spyker C8 Is the Quirkiest $250,000 Exotic Car in History                         |Doug DeMuro                         |1350802 |
|17.01.12     |Autos & Vehicles|2            |I just bought the most ridiculous car                                      

In [12]:
# Analyze trending rank distribution
print("Trending Rank Analysis:")

# Count videos by rank position
print("\nDistribution of trending ranks:")
df_with_rank.groupBy("trending_rank").count().orderBy("trending_rank").show(10)

# Show top-ranked videos across all categories and dates
print("\nTop-ranked videos (rank = 1) by category and date:")
top_ranked = df_with_rank.filter(F.col("trending_rank") == 1)
print(f"Total #1 ranked videos: {top_ranked.count()}")

top_ranked.select(
    "trending_date", "category_name", "title", "channel_title", "views", "engagement_score"
).orderBy(F.desc("views")).show(10, truncate=False)

Trending Rank Analysis:

Distribution of trending ranks:
+-------------+-----+
|trending_rank|count|
+-------------+-----+
|            1| 2864|
|            2| 2625|
|            3| 2481|
|            4| 2360|
|            5| 2215|
|            6| 2078|
|            7| 1989|
|            8| 1897|
|            9| 1774|
|           10| 1633|
+-------------+-----+
only showing top 10 rows


Top-ranked videos (rank = 1) by category and date:
Total #1 ranked videos: 2864
+-------------+-------------+---------------------------------------------------+-------------------+---------+--------------------+
|trending_date|category_name|title                                              |channel_title      |views    |engagement_score    |
+-------------+-------------+---------------------------------------------------+-------------------+---------+--------------------+
|18.02.06     |Music        |Childish Gambino - This Is America (Official Video)|ChildishGambinoVEVO|225211923|0.0121467938444804

## Final Dataset with All Features

Let's create the final dataset with all the engineered features and analyze the results.

In [13]:
# Create final dataset with all features
print("Creating final dataset with all engineered features...")

# Select relevant columns for the final dataset
final_df = df_with_rank.select(
    "video_id", "title", "channel_title", "category_name", "country",
    "trending_date", "publish_time", "views", "likes", "dislikes", "comment_count",
    "engagement_score", "days_to_trend", "trending_rank"
)

print(f"Final dataset contains {final_df.count():,} records with {len(final_df.columns)} columns")

# Show schema of final dataset
print("\nFinal dataset schema:")
final_df.printSchema()

Creating final dataset with all engineered features...
Final dataset contains 40,899 records with 14 columns

Final dataset schema:
root
 |-- video_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- views: long (nullable = true)
 |-- likes: long (nullable = true)
 |-- dislikes: long (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- engagement_score: double (nullable = true)
 |-- days_to_trend: integer (nullable = true)
 |-- trending_rank: integer (nullable = false)



In [14]:
# Show comprehensive sample of final dataset
print("Sample of final dataset with all engineered features:")
final_df.orderBy(F.desc("engagement_score")).show(10, truncate=False)

Sample of final dataset with all engineered features:
+-----------+-------------------------------------------------------------+------------------+-------------+-------+-------------+------------------------+-------+-------+--------+-------------+-------------------+-------------+-------------+
|video_id   |title                                                        |channel_title     |category_name|country|trending_date|publish_time            |views  |likes  |dislikes|comment_count|engagement_score   |days_to_trend|trending_rank|
+-----------+-------------------------------------------------------------+------------------+-------------+-------+-------------+------------------------+-------+-------+--------+-------------+-------------------+-------------+-------------+
|LsoLEjrDogU|Bruno Mars - Finesse (Remix) [Feat. Cardi B] [Official Video]|Bruno Mars        |Music        |US     |18.04.01     |2018-01-04T04:49:43.000Z|548621 |159356 |2374    |19455        |0.1567371646364248 |0  

In [15]:
# Summary statistics for all engineered features
print("Summary Statistics for Engineered Features:")
print("=" * 50)

# Engagement Score
print("\n1. Engagement Score:")
final_df.select(
    F.count("engagement_score").alias("count"),
    F.mean("engagement_score").alias("mean"),
    F.min("engagement_score").alias("min"),
    F.max("engagement_score").alias("max")
).show()

# Days to Trend
print("\n2. Days to Trend:")
final_df.select(
    F.count("days_to_trend").alias("count"),
    F.mean("days_to_trend").alias("mean"),
    F.min("days_to_trend").alias("min"),
    F.max("days_to_trend").alias("max")
).show()

# Trending Rank
print("\n3. Trending Rank:")
final_df.select(
    F.count("trending_rank").alias("count"),
    F.mean("trending_rank").alias("mean"),
    F.min("trending_rank").alias("min"),
    F.max("trending_rank").alias("max")
).show()

Summary Statistics for Engineered Features:

1. Engagement Score:
+-----+-------------------+---+------------------+
|count|               mean|min|               max|
+-----+-------------------+---+------------------+
|40899|0.01886712558920193|0.0|0.1567371646364248|
+-----+-------------------+---+------------------+


2. Days to Trend:
+-----+-----------------+---+----+
|count|             mean|min| max|
+-----+-----------------+---+----+
|40899|16.82757524633854|  0|4215|
+-----+-----------------+---+----+


3. Trending Rank:
+-----+------------------+---+---+
|count|              mean|min|max|
+-----+------------------+---+---+
|40899|13.359128585050978|  1| 61|
+-----+------------------+---+---+



In [16]:
# Clean up
SparkUtils.stop_spark_session()
print("Spark session stopped.")

Spark session stopped.
